# Pruebas de Modelo
Se van a realizar algunas pruebas sobre la red neuronal siamesa creada anteriormente. Este modelo se encuentra almacenado en una carpeta de Google Drive para poder utilizar en el notebook actual.

Para las pruebas se van a utilizar combinaciones de imágenes aleatorias para observar la similitud entre las dos entradas determinada por el modelo cargado.

## Importaciones

In [1]:
from google.colab import drive
from os import listdir
from os.path import isfile, join
from scipy.io import loadmat
from random import randint

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.backend as K
import pandas as pd
import numpy as np
import random

import datetime
import os

## Obtención de Datos

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_datos_dir = 'drive/MyDrive/DATOS_EXPERIMENTALES_JACKET/DATOS'

In [4]:
dataset_datos_files = [dataset_datos_dir+'/'+filename for filename in listdir(dataset_datos_dir) if isfile(join(dataset_datos_dir, filename))]

In [5]:
datos_fisura = []
datos_pernoflojo = []

In [6]:
for filename in listdir(dataset_datos_dir):
  if isfile(join(dataset_datos_dir, filename)):
    nombre_archivo = filename.split('.')[0]
    if (nombre_archivo.split('_')[0] == '3'):
      datos_fisura.append(dataset_datos_dir + '/' + filename)
    elif (nombre_archivo.split('_')[0] == '4'):
      datos_pernoflojo.append(dataset_datos_dir + '/' + filename)

### Datos Fisura

In [7]:
numero_experimentos_fisura = []
amplitud_experimentos_fisura = []

In [8]:
for nombre_archivo in datos_fisura:
  nombre_experimento = nombre_archivo.split('/')[4]
  numero_experimento = nombre_experimento.split('_')[1]
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  numero_experimentos_fisura.append(int(numero_experimento))
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_fisura.append(amplitud_experimento)
  else:
    amplitud_experimentos_fisura.append(int(amplitud_experimento))

In [9]:
df_fisura = pd.DataFrame()

In [10]:
for indice in range(len(datos_fisura)):
  direccion = datos_fisura[indice]
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  df['#_exp'] = numero_experimentos_fisura[indice]
  df['amplitud'] = amplitud_experimentos_fisura[indice]
  df_fisura = pd.concat([df_fisura, df], axis = 0)

In [11]:
df_fisura.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000132,0.000185,0.000122,0.000105,0.000141,0.000075,0.000007,0.000231,0.000299,0.000113,0.000228,0.000225,0.000168,0.000018,0.000344,0.000275,-0.000017,0.000046,0.000200,0.000169,0.000097,0.000196,-0.000053,0.000018,1,1.0
1,0.000092,0.000223,0.000166,0.000127,0.000191,0.000137,0.000075,0.000192,0.000240,0.000095,0.000279,0.000231,0.000145,0.000037,0.000339,0.000092,0.000085,0.000074,0.000128,0.000163,0.000074,0.000085,0.000063,-0.000027,1,1.0
2,0.000091,0.000280,0.000188,0.000130,0.000172,0.000102,0.000161,0.000158,0.000211,0.000123,0.000269,0.000206,0.000198,0.000041,0.000312,0.000179,0.000011,0.000079,0.000117,0.000146,0.000085,0.000089,-0.000014,0.000020,1,1.0
3,0.000179,0.000199,0.000249,0.000102,0.000268,0.000140,0.000126,0.000231,0.000239,0.000102,0.000284,0.000175,0.000164,0.000082,0.000311,0.000235,0.000031,0.000079,0.000257,0.000110,0.000050,0.000120,0.000080,0.000057,1,1.0
4,0.000282,0.000277,0.000202,0.000211,0.000251,0.000159,0.000096,0.000205,0.000342,0.000113,0.000240,0.000272,0.000214,0.000057,0.000304,0.000302,0.000047,0.000150,0.000271,0.000168,0.000126,0.000224,0.000105,0.000058,1,1.0
5,0.000005,0.000196,0.000247,0.000109,0.000186,0.000115,0.000042,0.000129,0.000255,0.000108,0.000315,0.000214,0.000237,-0.000001,0.000334,0.000202,-0.000008,0.000081,0.000173,0.000165,0.000126,0.000107,-0.000012,0.000006,1,1.0
6,0.000176,0.000372,0.000223,0.000154,0.000258,0.000165,0.000170,0.000157,0.000228,0.000126,0.000282,0.000219,0.000198,0.000081,0.000333,0.000146,0.000065,0.000070,0.000140,0.000145,0.000099,0.000115,0.000068,-0.000031,1,1.0
7,0.000133,0.000171,0.000187,0.000182,0.000237,0.000159,0.000121,0.000280,0.000293,0.000096,0.000248,0.000202,0.000160,0.000015,0.000319,0.000335,-0.000004,0.000102,0.000201,0.000193,0.000180,0.000171,-0.000009,0.000095,1,1.0


In [12]:
len(df_fisura)/99097

20.0

In [13]:
experimentos = [1, 2, 3, 4, 5]
wns = [0.5, 1, 2, 3]
arreglo_imagenes_por_experimento_fisura = []

for i in experimentos:
  for j in wns:
    filter1 = df_fisura["#_exp"] == i
    filter2 = df_fisura["amplitud"] == j
    dataset_experimento = df_fisura.where(filter1 & filter2).dropna()
    arreglo_matrices_asociadas = []

    for n in range(24):
      indice_inicio = 0
      indice_fin = 256
      columna_cada_6 = dataset_experimento[n][::6]
      matrices_columna = []
    
      while indice_fin < len(columna_cada_6):
        vector = columna_cada_6[indice_inicio:indice_fin]
        matriz = vector.to_numpy().reshape((16, 16))
        matrices_columna.append(matriz)
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      arreglo_matrices_asociadas.append(matrices_columna)
    
    arreglo_imagenes_por_experimento_fisura.append(arreglo_matrices_asociadas)

In [14]:
len(arreglo_imagenes_por_experimento_fisura)

20

In [15]:
len(arreglo_imagenes_por_experimento_fisura[0])

24

In [16]:
len(arreglo_imagenes_por_experimento_fisura[0][0])

64

In [17]:
arreglo_imagenes_por_experimento_fisura[0][0][0].size

256

In [18]:
arreglo_imagenes_por_experimento_fisura[0][0][0].shape

(16, 16)

In [19]:
arreglo_imagenes_fisura = []

for experimento in arreglo_imagenes_por_experimento_fisura:
  for idx in range(64):
    imagen = []
    for fila in experimento:
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    arreglo_imagenes_fisura.append(np.array(imagen))

In [20]:
len(arreglo_imagenes_fisura)

1280

In [21]:
indice_imagen_fisura = randint(0, len(arreglo_imagenes_fisura) - 1)
arreglo_imagenes_fisura[indice_imagen_fisura].shape, type(arreglo_imagenes_fisura[0]), indice_imagen_fisura

((24, 16, 16), numpy.ndarray, 513)

### Datos Perno Flojo

In [22]:
numero_experimentos_pernoflojo = []
amplitud_experimentos_pernoflojo = []

In [23]:
for nombre_archivo in datos_pernoflojo:
  nombre_experimento = nombre_archivo.split('/')[4]
  numero_experimento = nombre_experimento.split('_')[1]
  amplitud_experimento = nombre_experimento.split('.')[0].split('_')[2].replace('A', '')
  numero_experimentos_pernoflojo.append(int(numero_experimento))
  if amplitud_experimento == '05':
    amplitud_experimento = 0.5
    amplitud_experimentos_pernoflojo.append(amplitud_experimento)
  else:
    amplitud_experimentos_pernoflojo.append(int(amplitud_experimento))

In [24]:
df_pernoflojo = pd.DataFrame()

In [25]:
for indice in range(len(datos_pernoflojo)):
  direccion = datos_pernoflojo[indice]
  mat = loadmat(direccion)
  df = pd.DataFrame(mat['data'])
  df['#_exp'] = numero_experimentos_pernoflojo[indice]
  df['amplitud'] = amplitud_experimentos_pernoflojo[indice]
  df_pernoflojo = pd.concat([df_pernoflojo, df], axis = 0)

In [26]:
df_pernoflojo.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,#_exp,amplitud
0,0.000168,0.000204,0.000213,0.000135,0.000201,0.000099,0.000108,0.000189,0.000212,0.000093,0.000257,0.000196,0.000173,0.000029,0.000316,0.000202,0.000010,0.000085,0.000151,0.000136,0.000116,0.000099,0.000054,0.000111,1,0.5
1,0.000122,0.000277,0.000213,0.000180,0.000230,0.000172,0.000068,0.000213,0.000214,0.000095,0.000272,0.000172,0.000173,0.000050,0.000292,0.000178,0.000021,0.000022,0.000203,0.000139,0.000076,0.000164,0.000089,0.000076,1,0.5
2,0.000099,0.000214,0.000128,0.000153,0.000217,0.000100,0.000064,0.000174,0.000225,0.000116,0.000245,0.000202,0.000140,0.000012,0.000309,0.000256,0.000023,0.000089,0.000225,0.000162,0.000098,0.000154,0.000091,0.000092,1,0.5
3,0.000150,0.000258,0.000209,0.000163,0.000213,0.000120,0.000066,0.000157,0.000220,0.000096,0.000229,0.000203,0.000157,0.000057,0.000292,0.000260,0.000062,0.000077,0.000212,0.000178,0.000123,0.000150,0.000056,0.000130,1,0.5
4,0.000194,0.000244,0.000194,0.000201,0.000202,0.000127,0.000133,0.000197,0.000212,0.000110,0.000266,0.000199,0.000181,0.000040,0.000336,0.000209,0.000092,0.000097,0.000183,0.000170,0.000114,0.000127,0.000064,0.000087,1,0.5
5,0.000156,0.000287,0.000188,0.000119,0.000234,0.000127,0.000102,0.000198,0.000216,0.000097,0.000280,0.000222,0.000140,0.000058,0.000296,0.000212,0.000061,0.000097,0.000231,0.000156,0.000090,0.000155,0.000076,0.000091,1,0.5
6,0.000150,0.000230,0.000198,0.000239,0.000247,0.000150,0.000052,0.000178,0.000255,0.000115,0.000235,0.000189,0.000216,0.000072,0.000299,0.000233,0.000060,0.000124,0.000225,0.000157,0.000133,0.000153,0.000060,0.000087,1,0.5
7,0.000130,0.000230,0.000207,0.000116,0.000212,0.000108,0.000096,0.000178,0.000200,0.000130,0.000260,0.000203,0.000216,0.000057,0.000329,0.000261,0.000031,0.000045,0.000236,0.000178,0.000081,0.000160,0.000079,0.000115,1,0.5


In [27]:
len(df_pernoflojo)/99097

20.0

In [28]:
arreglo_imagenes_por_experimento_pernoflojo = []

for i in experimentos:
  for j in wns:
    filter1 = df_pernoflojo["#_exp"] == i
    filter2 = df_pernoflojo["amplitud"] == j
    dataset_experimento = df_pernoflojo.where(filter1 & filter2).dropna()
    arreglo_matrices_asociadas = []

    for n in range(24):
      indice_inicio = 0
      indice_fin = 256
      columna_cada_6 = dataset_experimento[n][::6]
      matrices_columna = []
    
      while indice_fin < len(columna_cada_6):
        vector = columna_cada_6[indice_inicio:indice_fin]
        matriz = vector.to_numpy().reshape((16, 16))
        matrices_columna.append(matriz)
        indice_inicio = indice_fin
        indice_fin = indice_fin + 256
      
      arreglo_matrices_asociadas.append(matrices_columna)
    
    arreglo_imagenes_por_experimento_pernoflojo.append(arreglo_matrices_asociadas)

In [29]:
len(arreglo_imagenes_por_experimento_pernoflojo)

20

In [30]:
len(arreglo_imagenes_por_experimento_pernoflojo[0])

24

In [31]:
len(arreglo_imagenes_por_experimento_pernoflojo[0][0])

64

In [32]:
arreglo_imagenes_por_experimento_pernoflojo[0][0][0].size

256

In [33]:
arreglo_imagenes_por_experimento_pernoflojo[0][0][0].shape

(16, 16)

In [34]:
arreglo_imagenes_pernoflojo = []

for experimento_i in arreglo_imagenes_por_experimento_pernoflojo:
  for idx in range(64):
    imagen = []
    for fila in experimento_i:
      capa_imagen = fila[idx]
      imagen.append(capa_imagen)

    arreglo_imagenes_pernoflojo.append(np.array(imagen))

In [35]:
len(arreglo_imagenes_pernoflojo)

1280

In [36]:
indice_imagen_pernoflojo = randint(0, len(arreglo_imagenes_pernoflojo) - 1)
arreglo_imagenes_pernoflojo[indice_imagen_pernoflojo].shape, type(arreglo_imagenes_pernoflojo[0]), indice_imagen_pernoflojo

((24, 16, 16), numpy.ndarray, 861)

### Carga del Modelo
Cargamos el modelo que se guardó previamente en Google Drive.

In [37]:
def load_model(model_path):
  """
  Carga un modelo guardado previamente.
  """
  print(f'Cargando modelo guardado de: {model_path}...')
  model = tf.keras.models.load_model(model_path)

  return model

In [39]:
# Almacenamos el modelo en una variable
modelo_7_loaded = load_model('drive/MyDrive/modelos/20210626-19311624735904-modelo_SNN_TurbinaJacket_fisura_pernoflojo.h5')

Cargando modelo guardado de: drive/MyDrive/modelos/20210626-19311624735904-modelo_SNN_TurbinaJacket_fisura_pernoflojo.h5...


## Pruebas
Para las pruebas se va a crear una función que sea capaz de seleccionar las siguientes combinaciones de imágenes:
- fisura / fisura
- perno flojo / perno flojo
- fisura / perno flojo

En la función sólo se debe ingresar:
- el modelo entrenado
- el arreglo de imágenes de fisura
- el arreglo de imágenes de perno flojo

Para aleatorizar las pruebas se establece un "modo", el cual puede ser 1, 2 o 3 por medio de una función aleatoria de la librería random. El modo 1, el modo 2 y el modo 3 se asocian con las combinaciones de imágenes mencionadas previamente.

In [126]:
def prueba_prediccion(modelo, arreglo_img1_fisura, arreglo_img2_pernoflojo):

  # Establecemos el modo de manera aleatoria
  modo = random.randint(1, 3)

  # Inicializamos las imágenes de prueba
  imagen_prueba_1 = []
  imagen_prueba_2 = []

  # Establecemos las condiciones de modo para generar el par de imágenes a comparar
  if modo == 1:
    imagen_prueba_1 = random.choice(arreglo_img1_fisura)
    imagen_prueba_2 = random.choice(arreglo_img1_fisura)
  elif modo == 2:
    imagen_prueba_1 = random.choice(arreglo_img2_pernoflojo)
    imagen_prueba_2 = random.choice(arreglo_img2_pernoflojo)
  elif modo == 3:
    imagen_prueba_1 = random.choice(arreglo_img1_fisura)
    imagen_prueba_2 = random.choice(arreglo_img2_pernoflojo)

  # Expandimos dimensiones para poder ingresar las imágenes al modelo (relacionado a número de batches)
  imagen_prueba_1 = np.expand_dims(imagen_prueba_1, axis=-1)
  imagen_prueba_2 = np.expand_dims(imagen_prueba_2, axis=-1)

  imagen_prueba_1 = np.expand_dims(imagen_prueba_1, axis=0)
  imagen_prueba_2 = np.expand_dims(imagen_prueba_2, axis=0)

  # Realizamos predicción
  prediccion_prueba = modelo.predict([imagen_prueba_1, imagen_prueba_2])[0][0]
  prediccion_prueba_redondeada = round(modelo.predict([imagen_prueba_1, imagen_prueba_2])[0][0])

  # Establecemos condiciones para mostrar distintos mensajes de acuerdo a la combinación escogida aleatoriamente
  if modo == 1:
    print("---------- Resultados ----------")
    print("Primera imagen: fisura")
    print("Segunda imagen: fisura")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1  else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 2:
    print("---------- Resultados ----------")
    print("Primera imagen: perno flojo")
    print("Segunda imagen: perno flojo")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")
  if modo == 3:
    print("---------- Resultados ----------")
    print("Primera imagen: fisura")
    print("Segunda imagen: perno flojo")
    print("Predicción: similares") if prediccion_prueba_redondeada == 1 else print("Predicción: no similares")
    print("Descripción real: no similares")
    print(f"Porcentaje: '{prediccion_prueba*100:.2f}%")

In [136]:
# Realizamos pruebas
prueba_prediccion(modelo=modelo_7_loaded, arreglo_img1_fisura=arreglo_imagenes_fisura, arreglo_img2_pernoflojo=arreglo_imagenes_pernoflojo)

---------- Resultados ----------
Primera imagen: fisura
Segunda imagen: fisura
Predicción: similares
Descripción real: similares
Porcentaje: '77.34%
